In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs
from qiskit_metal.qlibrary.qubits.transmon_pocket_cl import TransmonPocketCL
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.lumped.cap_3_interdigital import Cap3Interdigital
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.qubits.transmon_cross_fl import TransmonCrossFL
from qiskit_metal.qlibrary.qubits.transmon_pocket_cl import TransmonPocketCL
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.lumped.cap_3_interdigital import Cap3Interdigital
from qiskit_metal.qlibrary.qubits.transmon_pocket_cl import TransmonPocketCL
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled

In [3]:
design = designs.DesignPlanar()
gui = metal.MetalGUI(design)
gui = metal.MetalGUI(design)

string1 = "hello"
string1 = "Justin"

In [4]:
design.overwrite_enabled = True

In [5]:
design.chips.main.size.size_x = '11mm'
design.chips.main.size.size_y = '9mm'

In [6]:
design.chips.main

{'material': 'silicon',
 'layer_start': '0',
 'layer_end': '2048',
 'size': {'center_x': '0.0mm',
  'center_y': '0.0mm',
  'center_z': '0.0mm',
  'size_x': '11mm',
  'size_y': '9mm',
  'size_z': '-750um',
  'sample_holder_top': '890um',
  'sample_holder_bottom': '1650um'}}

In [7]:
options = dict(
    pad_width = '425 um',
    pocket_height = '650um',
    
     connection_pads=dict( 
        a = dict(loc_W=+1,loc_H=+1, pad_width='100um'), 
        b = dict(loc_W=-1,loc_H=-1, pad_height='30um', orientation='180'),
     )
)

q1 = TransmonPocket6(design, 'Q1', options = dict(
        pos_x='+.6mm', pos_y='0mm', orientaion = '90', hfss_inductance ='14nH', **options))

q2 = TransmonPocket6(design, 'Q2', options = dict(
        pos_x='-.6mm', pos_y='0mm', orientaion = '90', hfss_inductance ='12nH', **options))

gui.rebuild()
gui.autoscale()

In [8]:
TransmonPocket6.get_template_options(design)

{'pos_x': '0um',
 'pos_y': '0um',
 'connection_pads': {},
 '_default_connection_pads': {'pad_gap': '15um',
  'pad_width': '125um',
  'pad_height': '30um',
  'pad_cpw_shift': '0um',
  'pad_cpw_extent': '25um',
  'cpw_width': '10um',
  'cpw_gap': '6um',
  'cpw_extend': '100um',
  'pocket_extent': '5um',
  'pocket_rise': '0um',
  'loc_W': '+1',
  'loc_H': '+1'},
 'chip': 'main',
 'pad_gap': '30um',
 'inductor_width': '20um',
 'pad_width': '455um',
 'pad_height': '90um',
 'pocket_width': '650um',
 'pocket_height': '650um',
 'orientation': '0',
 'gds_cell_name': 'my_other_junction'}

In [9]:

coupler_options = Dict(
                        pin_inputs=Dict(
                                start_pin=Dict(
                                    component='Q1', pin='b'),
                        end_pin=Dict(
                                component='Q2',
                                pin='a')),
                        fillet='99.9um',
                        total_length = '3mm',
                        lead = Dict(
                            start_straight = '150um',
                            end_straight = '200um'
                                    )
)

bus = RoutePathfinder(design, 'coupler', options= coupler_options)



gui.rebuild()
gui.autoscale()

In [10]:
cap_Q1 = Cap3Interdigital(design, 'Cap_Q1', options= dict(pos_x='2.5mm', pos_y='0.25mm', orientation='90', finger_length = '40um'))
cap_Q2 = Cap3Interdigital(design, 'Cap_Q2', options= dict(pos_x='-2.5mm', pos_y='-0.25mm', orientation='-90', finger_length = '40um'))

gui.rebuild()
gui.autoscale()

In [11]:
readout_Q1 = RouteMeander(design,'Readout_Q1', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q1',
                                                    pin='a'),
                                                end_pin=Dict(
                                                    component='Cap_Q1',
                                                    pin='a')
                                            ),
                                            lead=Dict(
                                                start_straight='0.100mm',
                                                end_straight = '125um'
                                            ),
                                            meander=Dict(
                                                asymmetry = '-50um'),
                                            fillet = "99um",
                                            total_length = '6mm'))

gui.rebuild()
gui.autoscale()

In [12]:
readout_Q2 = RouteMeander(design,'Readout_Q2', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q2',
                                                    pin='b'),
                                                end_pin=Dict(
                                                    component='Cap_Q2',
                                                    pin='a')
                                            ),
                                            lead=Dict(
                                                start_straight='.125mm',
                                                end_straight = '125um'
                                            ),
                                            meander=Dict(
                                                asymmetry = '-150um'),
                                            fillet = "99um",
                                            total_length = '6mm'))

gui.rebuild()
gui.autoscale()

In [13]:
launch_Q1_read = LaunchpadWirebond(design, 'Launch_Q1_Read', options = dict(pos_x = '4.5mm', orientation = '180'))
launch_Q2_read = LaunchpadWirebond(design, 'Launch_Q2_Read', options = dict(pos_x = '-3.5mm', orientation = '0'))


gui.rebuild()
gui.autoscale()

In [14]:
tl_Q1 = RoutePathfinder(design, 'TL_Q1', options = dict(
                                            fillet='99um',
                                            lead=dict(end_straight='150um'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Launch_Q1_Read',
                                                    pin='tie'),
                                                end_pin=Dict(
                                                    component='Cap_Q1',
                                                    pin='b')
                                            )))

tl_Q2 = RoutePathfinder(design, 'TL_Q2', options = dict(
                                            fillet='99um',
                                            lead=dict(end_straight='150um'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Launch_Q2_Read',
                                                    pin='tie'),
                                                end_pin=Dict(
                                                    component='Cap_Q2',
                                                    pin='b')
                                            )))

gui.rebuild()
gui.autoscale()

In [15]:
gui = MetalGUI(design)

In [16]:
q0_q3d = design.renderers.q3d
q0_q3d.open_ansys() 
q0_q3d.new_ansys_project()

AttributeError: 'QQ3DRenderer' object has no attribute 'open_ansys'

In [ ]:
q0_q3d.connect_ansys()

In [ ]:
q0_q3d.activate_q3d_design("Qubit1")
q0_q3d.add_q3d_setup(name = 'QubitTune', max_passes = 15, min_converged_passes = 2, percent_error = 0.1)
q0_q3d.activate_q3d_setup('QubitTune')

In [ ]:
q0_q3d.render_design(['Q1'], [('Q1', 'readout'), ('Q1', 'bus')])

In [ ]:
q0_q3d.analyze_setup("QubitTune")

In [ ]:
q0_q3d.get_capacitance_matrix()

In [ ]:
dict_lom = q0_q3d.lumped_oscillator_vs_passes(14, 2, 2, 7, [0.1, 0.1], 9)

In [ ]:
q0_q3d.plot_convergence_main(dict_lom);
q0_q3d.plot_convergence_chi(dict_lom)

In [ ]:
q0_q3d.disconnect_ansys()